In [1]:
import tensorflow as tf
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import cross_validation
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
white = np.loadtxt('/0822/TensorFlow-Tutorials/white.csv', skiprows=1, delimiter=',', dtype=np.float32)
x_data = white[:, 0:-1]; y_data = white[:, [-1]]
Y_binary = (y_data > 5)
def minmaxscaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / denominator
X_mm = minmaxscaler(x_data)
X_train, X_test, Y_train, Y_test = train_test_split(X_mm,Y_binary, test_size= 0.3, random_state= 1)

In [3]:
X = tf.placeholder(tf.float32, [None, len(X_train[0])])
Y = tf.placeholder(tf.float32, [None, 1])

In [4]:
def mlp(layer_n, data, output_size, act='None'):
    w = tf.get_variable(layer_n+'w',[data.get_shape().as_list()[1],output_size],initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(layer_n+'b',[output_size],initializer=tf.contrib.layers.xavier_initializer())
    if act == 'relu':
        res = tf.nn.relu(tf.matmul(data,w)+b)
        return res
    elif act == 'sigmoid':
        res = tf.nn.sigmoid(tf.matmul(data,w)+b)
        return res
    else:
        return tf.matmul(data,w)+b

In [5]:
keep_prob = tf.placeholder(tf.float32)
l1=tf.nn.dropout(mlp('l1', X, 22, 'relu'), keep_prob)
l2=tf.nn.dropout(mlp('l2', l1, 22, 'relu'), keep_prob)
l3=tf.nn.dropout(mlp('l3', l2, 44, 'relu'), keep_prob)
l4=tf.nn.dropout(mlp('l4', l3, 44, 'relu'), keep_prob)
H=tf.nn.dropout(mlp('l5', l4, 1), keep_prob)

In [7]:
cost = tf.reduce_sum(tf.pow(H-Y,2))/len(X_train)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
predict = tf.cast(H > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.float32))

In [8]:
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

for step in range(0,10000):
    cost_val, _ = sess.run([cost, optimizer], feed_dict={X: X_train, Y: Y_train, keep_prob: 0.75})
    if(step % 200 == 0):
        print(step, cost_val)

print("Accuracy : ", sess.run(accuracy, feed_dict={X: X_test, Y: Y_test, keep_prob: 1.0}))

0 0.39620465


200 0.30648714


400 0.29065996


600 0.2869205


800 0.28545156


...

9200 0.2736646


9400 0.26238635


9600 0.26519153


9800 0.26304737


Accuracy :  0.7632653
